# Constants


In [11]:

import pandas as pd
import os
import shutil
from os import path as osp

import torch
from tiatoolbox.utils.misc import select_device
import random
import numpy as np
from pathlib import Path
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import joblib
import json
import glob

# from src.intensity import add_features_and_create_new_dicts

from src.featureextraction import get_cell_features, add_features_and_create_new_dicts
from src.train import stratified_split, recur_find_ext, run_once, rm_n_mkdir ,reset_logging
from src.graph_construct import create_graph_with_pooled_patch_nodes, get_pids_labels_for_key


ON_GPU = False
device = select_device(on_gpu=ON_GPU)

SEED = 5
random.seed(SEED)
rng = np.random.default_rng(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)


BASEDIR = '/home/amrit/data/proj_data/MLG_project/DLBCL-Morph'


STAIN = 'MYC'
# STAIN = 'BCL2'


FIDIR = f'{BASEDIR}/outputs'
CLINPATH = f'{BASEDIR}/clinical_data_cleaned.csv'
ANNPATH = f'{BASEDIR}/annotations_clean.csv'
FEATSDIR = f'{BASEDIR}/outputs/files/{STAIN}'
FEATSCALERPATH = f"{FEATSDIR}/0_feat_scaler.npz"
PATCH_SIZE = 224
OUTPUT_SIZE = PATCH_SIZE*8

WORKSPACE_DIR = Path(BASEDIR)
# GRAPH_DIR = WORKSPACE_DIR / f"graphs{STAIN}" 
# LABELS_PATH = WORKSPACE_DIR / "graphs/0_labels.txt"


# Graph construction
# PATCH_SIZE = 300
SKEW_NOISE = 0.0001
MIN_CELLS_PER_PATCH = 10
CONNECTIVITY_DISTANCE = 500


GRAPHSDIR = Path(f'{BASEDIR}/graphs/{STAIN}')
LABELSPATH = f'{BASEDIR}/graphs/{STAIN}_labels.json'

NUM_EPOCHS = 100
NUM_NODE_FEATURES = 128
NCLASSES = 3

# pooling= "mean"
# conv= "EdgeConv"
# aggr= "max"
# dropout  =  0.3
# layers = [64, 32, 32]


pooling= "mean"
conv= "EdgeConv"
aggr= "max"
dropout  =  0.3
layers = [64, 32, 32]


pooling= "mean"
conv= "GINConv"
aggr= "max"
dropout  =  0.3
layers = [64, 32, 32]


pooling= "mean"
conv= "GINConv"
aggr= "max"
dropout  =  0.3
layers = [64, 32, 32]

arch_kwargs = {
    "dim_features": NUM_NODE_FEATURES,
    "dim_target": NCLASSES,
    "layers": layers,
    "dropout": dropout,
    "pooling": pooling,
    "conv": conv,
    "aggr": aggr,
}


TRAIN_DIR = WORKSPACE_DIR / "training"
SPLIT_PATH = TRAIN_DIR / f"splits_{STAIN}.dat"
# RUN_OUTPUT_DIR = TRAIN_DIR / f"session_{STAIN}_{conv}_{pooling}_{aggr}_{str(dropout)}_{str(layers)}_{datetime.now().strftime('%m_%d_%H_%M')}"
RUN_OUTPUT_DIR = TRAIN_DIR / f"session_{STAIN}_{conv}_{pooling}_{aggr}_{str(dropout)}_{str(layers)}"

RUN_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR = RUN_OUTPUT_DIR / "model"

In [12]:
wsi_paths = recur_find_ext(GRAPHSDIR, [".json"])
wsi_names = [Path(v).stem for v in wsi_paths]
assert len(wsi_paths) > 0, "No files found."  # noqa: S101

len(wsi_paths) , len(wsi_names) , wsi_names[:5]

(319,
 319,
 ['13901_MYC_532',
  '13901_MYC_557',
  '13902_MYC_513',
  '13902_MYC_514',
  '13903_MYC_535'])

In [13]:
NUM_FOLDS = 1
TEST_RATIO = 0.2
TRAIN_RATIO = 0.8 * 0.7
VALID_RATIO = 0.8 * 0.3

# if SPLIT_PATH and os.path.exists(SPLIT_PATH):
#     splits = joblib.load(SPLIT_PATH)
# else:
x = np.array(wsi_names)
with open(LABELSPATH, 'r') as f:
    labels_dict = json.load(f)
print(labels_dict)
y = np.array([labels_dict[wsi_name+'.json'] for wsi_name in wsi_names])
y[np.where(y==-1)] = 0
splits = stratified_split(x, y, TRAIN_RATIO, VALID_RATIO, TEST_RATIO, NUM_FOLDS)
joblib.dump(splits, SPLIT_PATH)


{'13901_MYC_532.json': 0, '13901_MYC_557.json': 0, '13902_MYC_513.json': 2, '13902_MYC_514.json': 2, '13903_MYC_535.json': 2, '13903_MYC_560.json': 2, '13904_MYC_539.json': 1, '13904_MYC_550.json': 1, '13908_MYC_527.json': 1, '13908_MYC_555.json': 1, '13911_MYC_520.json': 2, '13911_MYC_533.json': 2, '13912_MYC_530.json': 0, '13913_MYC_536.json': 0, '13913_MYC_554.json': 0, '13914_MYC_534.json': 0, '13914_MYC_556.json': 0, '13915_MYC_516.json': 2, '13915_MYC_547.json': 2, '13917_MYC_526.json': 2, '13917_MYC_551.json': 2, '13919_MYC_518.json': 0, '13919_MYC_528.json': 0, '13920_MYC_549.json': 2, '13922_MYC_558.json': 2, '13923_MYC_525.json': 2, '13923_MYC_538.json': 2, '13924_MYC_515.json': 1, '13924_MYC_548.json': 1, '13952_MYC_2587.json': 1, '13952_MYC_2589.json': 1, '13953_MYC_2611.json': 2, '13953_MYC_2639.json': 2, '13954_MYC_2592.json': 2, '13955_MYC_2637.json': 1, '13957_MYC_2646.json': 1, '13958_MYC_2600.json': 2, '13958_MYC_2640.json': 2, '13959_MYC_2593.json': 1, '13959_MYC_264

['/home/amrit/data/proj_data/MLG_project/DLBCL-Morph/training/splits_MYC.dat']

In [22]:
# # we must define the function after training/loading
# def nodes_preproc_func(node_features: np.ndarray) -> np.ndarray:
#     """Pre-processing function for nodes."""
#     return node_scaler.transform(node_features)
nodes_preproc_func = None


splits = joblib.load(SPLIT_PATH)
loader_kwargs = {
    "num_workers": 6,
    "batch_size": 32,
}

optim_kwargs = {
    "lr": 5.0e-4,
    "weight_decay": 1.0e-4,
}

NUM_EPOCHS = 100
# if not MODEL_DIR.exists() or True:
for split_idx, split in enumerate(splits):
        new_split = {
            "train": split["train"],
            "infer-train": split["train"],
            "infer-valid-A": split["valid"],
            "infer-valid-B": split["test"],
        }
        MODEL_DIR = Path(MODEL_DIR) 
        split_save_dir = MODEL_DIR / f"{split_idx:02d}/"
        rm_n_mkdir(split_save_dir)
        reset_logging(split_save_dir)
        run_once(
            new_split,
            NUM_EPOCHS,
            save_dir=split_save_dir,
            arch_kwargs=arch_kwargs,
            loader_kwargs=loader_kwargs,
            optim_kwargs=optim_kwargs,
            on_gpu=ON_GPU,
            GRAPH_DIR=GRAPHSDIR
        )

|2023-11-21|22:48:57.721| [INFO] EPOCH: 000


SlideGraphArch(
  (first_h): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (nns): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=64, out_features=32, bias=True)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): Sequential(
      (0): Linear(in_features=32, out_features=32, bias=True)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (convs): ModuleList(
    (0): GINConv(nn=Sequential(
      (0): Linear(in_features=64, out_features=32, bias=True)
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    ))
    (1): GINConv(nn=Sequential(
      (0): Linear(in_features=32, out_features=32, bias=True)
      (1): BatchNorm1d(32, eps=1e-05, mom

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:00<00:00, 14.82it/s]
|2023-11-21|22:48:58.061| [INFO] train-EMA-loss: 1.287249
|2023-11-21|22:48:58.342| [INFO] infer-train-accuracy: 0.371000
|2023-11-21|22:48:58.606| [INFO] infer-valid-A-accuracy: 0.364000
|2023-11-21|22:48:58.848| [INFO] infer-valid-B-accuracy: 0.312000
|2023-11-21|22:48:59.079| [INFO] EPOCH: 001


best_score {'infer-train-accuracy': 0, 'infer-valid-A-accuracy': 0, 'infer-valid-B-accuracy': 0}


...

best_score {'infer-train-accuracy': 0.702, 'infer-valid-A-accuracy': 0.455, 'infer-valid-B-accuracy': 0.594}


100%|██████████| 5/5 [00:00<00:00, 16.40it/s]
|2023-11-21|22:51:29.518| [INFO] train-EMA-loss: 0.465398
|2023-11-21|22:51:29.794| [INFO] infer-train-accuracy: 0.624000
|2023-11-21|22:51:30.044| [INFO] infer-valid-A-accuracy: 0.403000
|2023-11-21|22:51:30.291| [INFO] infer-valid-B-accuracy: 0.484000
|2023-11-21|22:51:30.554| [INFO] EPOCH: 099


best_score {'infer-train-accuracy': 0.702, 'infer-valid-A-accuracy': 0.455, 'infer-valid-B-accuracy': 0.594}


100%|██████████| 5/5 [00:00<00:00, 16.63it/s]
|2023-11-21|22:51:30.857| [INFO] train-EMA-loss: 0.285522
|2023-11-21|22:51:31.141| [INFO] infer-train-accuracy: 0.629000
|2023-11-21|22:51:31.392| [INFO] infer-valid-A-accuracy: 0.390000
|2023-11-21|22:51:31.637| [INFO] infer-valid-B-accuracy: 0.516000


best_score {'infer-train-accuracy': 0.702, 'infer-valid-A-accuracy': 0.455, 'infer-valid-B-accuracy': 0.594}
